In [ ]:
import luigi
import pandas as pd
import os


## Création du dossier pour stocker les données nettoyées

In [ ]:
# Vérifiez si le dossier n'existe pas déjà

dossier = 'CleanData'

if not os.path.exists(dossier):
    # Créez le dossier
    os.makedirs(dossier, mode=0o777)
    os.makedirs(os.path.join(dossier, "2017"), mode=0o777)
    os.makedirs(os.path.join(dossier, "2022"), mode=0o777)
    print("Dossier créé avec succès.")


## Extraire les données sur la délinquance

In [ ]:
class ExtractCommunes(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/2017/communes.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        
        # Chargement des dataset
        communes = pd.read_csv(self.file_path, sep=",", low_memory=False)
        df_partis = pd.read_csv('./RawData/Partis.csv', sep=",")

        # Sélectionner les colonnes nécessaires
        colonnes = ['Code du département', 'Libellé du département', 'Code de la commune', 'Libellé de la commune', 'Inscrits', 'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs', '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', 'Exprimés', '% Exp/Ins', '% Exp/Vot']

        # Créer un nouveau DataFrame pour les candidats distincts
        df_candidats = pd.DataFrame()

        # Parcourir les colonnes par groupes de 8 (car chaque candidat occupe 8 colonnes)
        for i in range(18, len(communes.columns), 7):
            # Sélectionner les colonnes du candidat actuel
            colonnes_candidat = communes.columns[i:i+7]

            # Renommer les colonnes du candidat
            df_candidat = communes[colonnes_candidat].copy()
            
            df_candidat = pd.DataFrame(df_candidat.values, columns=['N°Panneau','Sexe','Nom','Prenom','Voix','% Voix/Ins','% Voix/Exp'])

            # Ajouter les colonnes supplémentaires (Code du département, Libellé du département, etc.)
            df_candidat = pd.concat([communes[colonnes], df_candidat], axis=1)            
            
            # Ajouter les données du candidat au DataFrame des candidats distincts
            df_candidats = pd.concat([df_candidats, df_candidat])

        df_candidats = pd.merge(df_candidats, df_partis, on=['Nom', 'Prenom'], how='left')

        df_candidats = df_candidats.drop(["Abstentions","% Abs/Ins","Votants","% Vot/Ins","% Blancs/Ins","% Blancs/Vot","% Nuls/Ins","% Nuls/Vot","Exprimés","% Exp/Ins","% Exp/Vot", "N°Panneau","Sexe","Nom","Prenom", "% Voix/Ins","% Voix/Exp", "Inscrits", "Nuls"], axis=1)
        # Réinitialiser les index du DataFrame des candidats distincts
        df_candidats = df_candidats.reset_index(drop=True)

        df_candidats["codgeo"] = df_candidats["Code du département"].astype(str).str.zfill(2) + df_candidats["Code de la commune"].astype(str).str.zfill(3)

        # Enregistrer le DataFrame des candidats distincts dans un nouveau fichier CSV
        df_candidats.to_csv(self.output().path, index=False)




In [ ]:
class ExtractLogements(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/2017/logement.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        logement = pd.read_csv(self.file_path, sep=";", dtype ='str')

        #renomes les colonnes
        logement = logement.rename(columns={"CODGEO": "codgeo", "P17_RP_VOIT2P" : "Au moins deux voitures", "P17_MEN": "Nb menages"})
        
        logement["Taux Voitures Menage"] = logement["Au moins deux voitures"].astype(float) / logement["Nb menages"].astype(float)

        colonnes = ["codgeo", "Taux Voitures Menage"]  

        logement = logement[colonnes]

        logement['codgeo'] = logement['codgeo'].astype(str)

          
        logement.to_csv(self.output().path, index=False)


In [ ]:
class MergeCommuneLogements(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def requires(self):
        return [ExtractCommunes(file_path="./RawData/communes.csv", file_path1="./RawData/partis.csv"),
                ExtractLogements(file_path="./RawData/logements.csv")]
    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/data.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        logement = pd.read_csv(self.file_path, sep=",")
        communes = pd.read_csv(self.file_path1, sep=",")


        merge_data = pd.merge(communes, logement, on='codgeo', how='inner')
        merge_data = merge_data.drop_duplicates()
        merge_data.to_csv(self.output().path, index=False)

In [ ]:

class ExtractRevenus(luigi.Task):
    file_path = luigi.Parameter()

    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/revenus.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        revenus = pd.read_csv(self.file_path, sep=";", dtype ='str')
        
        #renomes les colonnes
        revenus = revenus.rename(columns={"med_disp" : "Revenus"})
        
        colonnes = ["codgeo", "Revenus"]

        revenus = revenus[colonnes]
        
        revenus['codgeo'] = revenus['codgeo'].astype(str)
                
        revenus.to_csv(self.output().path, index=False)


In [ ]:
class MergeRevenusCommunes(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def requires(self):
        return [MergeCommuneLogements(file_path="./CleanData/2017/communes.csv", file_path1="./CleanData/2017/logement.csv"), ExtractRevenus(file_path="./RawData/revenu_annuel.csv")]
    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/data.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        revenus = pd.read_csv(self.file_path, sep=",")
        communes = pd.read_csv(self.file_path1, sep=",")

        merge_data = pd.merge(communes, revenus, on='codgeo', how='inner')
        
        merge_data.to_csv(self.output().path, index=False)

In [ ]:
class ExtractEmploi(luigi.Task):
    file_path = luigi.Parameter()

    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/emploi.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        emploi = pd.read_csv(self.file_path, sep=";", dtype ='str')

        #renomes les colonnes
        emploi = emploi.rename(columns={"CODGEO": "codgeo", "P17_CHOM1564" : "Nombre de chômeurs de 15 à 64 ans", "P17_ACT1564": "Population Active", "P17_POP1564" : "Population"})
        
        emploi["Taux Chomage"] = emploi["Nombre de chômeurs de 15 à 64 ans"].astype(float) / emploi["Population Active"].astype(float) 

        emploi = emploi.dropna(subset=['Taux Chomage'])

        colonnes = ["codgeo", "Nombre de chômeurs de 15 à 64 ans", "Population Active", 'Taux Chomage']
        emploi = emploi[colonnes]
        
        emploi['codgeo'] = emploi['codgeo'].astype(str)

        
        emploi.to_csv(self.output().path, index=False)


In [ ]:
class MergeEmloiCommunes(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def requires(self):
        return [MergeRevenusCommunes(file_path="./CleanData/2017/revenus.csv", file_path1="./CleanData/2017/data.csv"), ExtractEmploi(file_path="./RawData/emploi.csv")]
    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/data.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        emploi = pd.read_csv(self.file_path, sep=",")
        communes = pd.read_csv(self.file_path1, sep=",")

        merge_data = pd.merge(communes, emploi, on='codgeo', how='inner')
        merge_data = merge_data.drop_duplicates()
        merge_data.to_csv(self.output().path, index=False)

In [ ]:
class ExtractPopulation(luigi.Task):
    file_path = luigi.Parameter()

    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/population.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        population = pd.read_csv(self.file_path, sep=";", dtype ='str')

        #renomes les colonnes
        population = population.rename(columns={"DEPCOM" :"codgeo", "PTOT" : "Population Total"})
        
        colonnes = ["codgeo", "Population Total"]

        population = population[colonnes]
        
        population.to_csv(self.output().path, index=False)

In [ ]:
class MergePopulationCommunes(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def requires(self):
        return [MergeEmloiCommunes(file_path="./CleanData/2017/emploi.csv", file_path1="./CleanData/2017/data.csv"), ExtractPopulation(file_path="./RawData/population.csv")]
    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/data.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        population = pd.read_csv(self.file_path, sep=",")
        communes = pd.read_csv(self.file_path1, sep=",")

        merge_data = pd.merge(communes, population, on='codgeo', how='inner')

        merge_data.to_csv(self.output().path, index=False)

In [ ]:
import numpy as np

class ExtractDiplomes(luigi.Task):
    file_path = luigi.Parameter()

    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/diplomes.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        # Chargement des dataset
        diplomes = pd.read_csv(self.file_path, sep=";", dtype ='str')
    
        #renomes les colonnes
        diplomes = diplomes.rename(columns={"CODGEO" :"codgeo", "P17_SCOL1824" : "Nombre de personnes scolarisées"})
        
        diplomes["Taux scolarisées"] = np.where(diplomes["P17_POP1824"].astype(float) != 0,
                                       diplomes["Nombre de personnes scolarisées"].astype(float) / diplomes["P17_POP1824"].astype(float),
                                       0)

        display(diplomes)
        
        colonnes = ["codgeo", "Taux scolarisées"]

        diplomes = diplomes[colonnes]
        
        diplomes.to_csv(self.output().path, index=False)

In [ ]:
class MergeDiplomesCommunes(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def requires(self):
        return [MergePopulationCommunes(file_path="./CleanData/2017/population.csv", file_path1="./CleanData/2017/data.csv"), ExtractDiplomes(file_path="./RawData/diplomes.csv")]
    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/data.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        diplomes = pd.read_csv(self.file_path, sep=",")
        communes = pd.read_csv(self.file_path1, sep=",")

        merge_data = pd.merge(communes, diplomes, on='codgeo', how='inner')

        merge_data.to_csv(self.output().path, index=False)

In [ ]:
import numpy as np

class ExtractEntreprises(luigi.Task):
    file_path = luigi.Parameter()

    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/entreprises.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        # Chargement des dataset
        entreprises = pd.read_csv(self.file_path, sep=";", dtype ='str')
    
        entreprises = entreprises.loc[entreprises["an"] == '2017']

        #renomes les colonnes
        entreprises = entreprises.rename(columns={"CODGEO" :"codgeo", "tx_crea_ent" : "Taux Creation Entreprise"})
        
        entreprises["Taux Creation Entreprise"] = entreprises["Taux Creation Entreprise"].str.replace(',', '.').astype(float)

        entreprises["Taux Creation Entreprise"] = entreprises["Taux Creation Entreprise"].fillna(0)

        colonnes = ["codgeo", "Taux Creation Entreprise"]

        entreprises = entreprises[colonnes]
        
        entreprises.to_csv(self.output().path, index=False)

In [ ]:
class MergeEntreprisesCommunes(luigi.Task):
    file_path = luigi.Parameter()
    file_path1 = luigi.Parameter()

    def requires(self):
        return [MergeDiplomesCommunes(file_path="./CleanData/2017/diplomes.csv", file_path1="./CleanData/2017/data.csv"), ExtractEntreprises(file_path="./RawData/entreprises.csv")]
    
    def output(self):
        return luigi.LocalTarget("./CleanData/2017/data.csv")


    def complete(self):
        return self.output().exists()
    
    def run(self):
        
        # Chargement des dataset
        entreprises = pd.read_csv(self.file_path, sep=",")
        communes = pd.read_csv(self.file_path1, sep=",")

        merge_data = pd.merge(communes, entreprises, on='codgeo', how='inner')

        merge_data.to_csv(self.output().path, index=False)

## Exécuter toutes les tâches

In [ ]:
class ReadAllcommunes(luigi.Task):
    def requires(self):
        return [MergeEntreprisesCommunes(file_path="./CleanData/2017/entreprises.csv", file_path1="./CleanData/2017/data.csv")]
    
    def run(self):
        print("lancement")

    def output(self):
        return luigi.LocalTarget('result.txt')

In [ ]:

config = luigi.configuration.get_config()
config.set('core', 'no_lock', 'False')

dossier = "./Cleancommunes/"
Restart = True

if Restart is False:
    # Parcourir tous les fichiers du dossier
    for fichier in os.listdir(dossier):
        chemin_fichier = os.path.join(dossier, fichier)
        # Supprimer le fichier
        os.remove(chemin_fichier)

luigi.build([ReadAllcommunes()], local_scheduler=False, no_lock=True)